In [90]:
import pandas as pd
import pandas_gbq
import plotly.express as px
import pandas_gbq
import sys, requests
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option("styler.format.thousands", ",")

In [138]:
#Change the date to the previous day
query = """
SELECT * FROM `perceptive-ivy-290216.nba_api_player_boxscore.player_box_96_onwards`
"""
project_id = "perceptive-ivy-290216"
df_bq_reg_season = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [274]:
#USE FOR ALL TIME DATA
df_bq2=df_bq_reg_season.copy()

#USE FOR SINGLE SEASON PLOTS
# current_year="2023-24"
# df_bq2=df_bq_reg_season[df_bq_reg_season["Season"]==current_year].copy()

df_bq2=df_bq2.fillna(0)
df_bq2.head()

,SEASON_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,Season
18639,22023,1627739,Kris Dunn,1610612762,UTA,Utah Jazz,0022300085,2023-10-27,UTA vs. LAC,W,19,2,7,0.286,0,2,0.0,0,0,0.0,0,1,1,3,1,0,3,6,4,8,1,2023-24
18640,22023,1627739,Kris Dunn,1610612762,UTA,Utah Jazz,0022300072,2023-10-25,UTA vs. SAC,L,16,3,6,0.5,0,1,0.0,2,4,0.5,0,1,1,2,2,0,4,0,8,-16,1,2023-24
18641,22023,1629659,Talen Horton-Tucker,1610612762,UTA,Utah Jazz,0022300072,2023-10-25,UTA vs. SAC,L,22,3,9,0.333,2,4,0.5,0,0,0.0,0,1,1,4,0,1,1,1,8,-5,1,2023-24
18642,22023,1629012,Collin Sexton,1610612762,UTA,Utah Jazz,0022300072,2023-10-25,UTA vs. SAC,L,20,2,9,0.222,0,2,0.0,5,7,0.714,0,3,3,0,0,0,3,3,9,-10,1,2023-24
18643,22023,203903,Jordan Clarkson,1610612762,UTA,Utah Jazz,0022300072,2023-10-25,UTA vs. SAC,L,31,8,16,0.5,2,5,0.4,6,8,0.75,2,2,4,6,0,0,2,4,24,-4,1,2023-24


In [275]:
df_bq2['FGMX']=df_bq2['FGA']-df_bq2['FGM']
df_bq2['FG3MX']=df_bq2['FG3A']-df_bq2['FG3M']
df_bq2['FG2M']=df_bq2['FGM']-df_bq2['FG3M']
df_bq2['FG2A']=df_bq2['FGA']-df_bq2['FG3A']
df_bq2['FG2_PCT']=df_bq2['FG2M']/df_bq2['FG2A']
df_bq2=df_bq2.round({'FG2_PCT': 3})

In [276]:
#Add Game Number for trended values plotting
df_bq2["Game Number"] = df_bq2.groupby("PLAYER_NAME")["GAME_DATE"].rank(method="dense", ascending=True)
df_bq2["Game Number"]=df_bq2["Game Number"].astype(int)

In [277]:
#Get cumulative stat for each player
df_bq2['Total PTS'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['PTS'].cumsum()
df_bq2['Total REB'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['REB'].cumsum()
df_bq2['Total DREB'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['DREB'].cumsum()
df_bq2['Total OREB'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['OREB'].cumsum()
df_bq2['Total AST'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['AST'].cumsum()
df_bq2['Total STL'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['STL'].cumsum()
df_bq2['Total BLK'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['BLK'].cumsum()
df_bq2['Total TOV'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['TOV'].cumsum()
df_bq2['Total PF'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['PF'].cumsum()
df_bq2['Total FGM'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['FGM'].cumsum()
df_bq2['Total FGA'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['FGA'].cumsum()
df_bq2['Total FG3M'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['FG3M'].cumsum()
df_bq2['Total FG3A'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['FG3A'].cumsum()
df_bq2['Total FGMX'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['FGMX'].cumsum()
df_bq2['Total FG3MX'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['FG3MX'].cumsum()
df_bq2['Total FTM'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['FTM'].cumsum()
df_bq2['Total FTA'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['FTA'].cumsum()
df_bq2['Total MIN'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['MIN'].cumsum()

In [278]:
#Get Totals For Each Category
df_totals=pd.DataFrame(df_bq2.groupby(by=['PLAYER_ID','PLAYER_NAME'])[['MIN', 'FGM',
       'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA','OREB', 'DREB', 'REB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS','PLUS_MINUS','FGMX','FG3MX','FG2M','FG2A']].sum()).reset_index()
#Get Game Counts for Each Player
df_count=pd.DataFrame(df_bq2.groupby(by=['PLAYER_ID','PLAYER_NAME'])[["GAME_ID"]].count()).reset_index()
df_count.columns=["PLAYER_ID","PLAYER_NAME","GP"]
df_totals=df_totals.merge(df_count, left_on=["PLAYER_NAME","PLAYER_ID"],right_on=["PLAYER_NAME","PLAYER_ID"])
df_totals.head()

,PLAYER_ID,PLAYER_NAME,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,FGMX,FG3MX,FG2M,FG2A,GP
0,2544,LeBron James,2508,685,1269,149,363,303,404,61,457,518,589,89,38,245,78,1822,221,584,214,536,906,71
1,101108,Chris Paul,1533,206,467,78,210,43,52,18,207,225,393,70,6,76,108,533,110,261,132,128,257,58
2,200768,Kyle Lowry,1691,163,377,98,250,63,75,34,160,194,253,59,21,85,146,487,2,214,152,65,127,60
3,200782,P.J. Tucker,487,18,50,13,35,3,3,28,57,85,16,16,7,8,54,52,-23,32,22,5,15,31
4,201142,Kevin Durant,2790,751,1436,168,407,362,423,41,454,495,378,69,91,244,133,2032,290,685,239,583,1029,75


In [279]:
#Add additional columns
df_totals['FG_PCT']=df_totals['FGM']/df_totals['FGA']
df_totals['FG3_PCT']=df_totals['FG3M']/df_totals['FG3A']
df_totals['FG2_PCT']=df_totals['FG2M']/df_totals['FG2A']
df_totals=df_totals.round({'FG_PCT': 3,'FG3_PCT': 3,'FG2_PCT': 3})

In [280]:
#Re-order the columns
df_totals=df_totals[['PLAYER_NAME', 'GP', 'MIN', 'FGM', 'FGMX', 'FGA','FG_PCT', 'FG2M', 'FG2A','FG2_PCT','FG3M', 
          'FG3MX', 'FG3A', 'FG3_PCT', 'FTM', 'FTA','OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]
# df_totals["MIN"]=df_totals["MIN"].apply(lambda x : "{:,}".format(x))
df_totals.head()

,PLAYER_NAME,GP,MIN,FGM,FGMX,FGA,FG_PCT,FG2M,FG2A,FG2_PCT,FG3M,FG3MX,FG3A,FG3_PCT,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS
0,LeBron James,71,2508,685,584,1269,0.54,536,906,0.592,149,214,363,0.41,303,404,61,457,518,589,89,38,245,78,1822,221
1,Chris Paul,58,1533,206,261,467,0.441,128,257,0.498,78,132,210,0.371,43,52,18,207,225,393,70,6,76,108,533,110
2,Kyle Lowry,60,1691,163,214,377,0.432,65,127,0.512,98,152,250,0.392,63,75,34,160,194,253,59,21,85,146,487,2
3,P.J. Tucker,31,487,18,32,50,0.36,5,15,0.333,13,22,35,0.371,3,3,28,57,85,16,16,7,8,54,52,-23
4,Kevin Durant,75,2790,751,685,1436,0.523,583,1029,0.567,168,239,407,0.413,362,423,41,454,495,378,69,91,244,133,2032,290


In [281]:
sort_category="PTS"

In [282]:
#Sort by a category and then rank it. This will help get the Top Players in a category
df_totals['RK_Category']=df_totals[sort_category].rank(method="dense", ascending=False)
df_totals['RK_Category']=df_totals['RK_Category'].astype(int)
top=df_totals.sort_values(by=sort_category, ascending=False).head(10)
top

,PLAYER_NAME,GP,MIN,FGM,FGMX,FGA,FG_PCT,FG2M,FG2A,FG2_PCT,FG3M,FG3MX,FG3A,FG3_PCT,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,RK_Category
224,Luka Doncic,70,2624,804,848,1652,0.487,520,908,0.573,284,460,744,0.382,478,608,59,587,646,686,99,38,282,149,2370,319,1
196,Shai Gilgeous-Alexander,75,2566,796,691,1487,0.535,701,1218,0.576,95,174,269,0.353,567,649,65,350,415,465,150,67,162,184,2254,608,2
73,Giannis Antetokounmpo,73,2573,837,532,1369,0.611,803,1245,0.645,34,90,124,0.274,514,782,196,645,841,476,87,79,250,210,2222,339,3
190,Jalen Brunson,77,2729,790,858,1648,0.479,579,1122,0.516,211,315,526,0.401,421,497,43,235,278,519,70,13,185,144,2212,512,4
97,Nikola Jokic,79,2737,822,589,1411,0.583,739,1180,0.626,83,148,231,0.359,358,438,223,753,976,708,108,68,237,194,2085,682,5
291,Anthony Edwards,79,2775,718,840,1558,0.461,528,1026,0.515,190,342,532,0.357,423,506,52,378,430,405,101,42,241,141,2049,430,6
4,Kevin Durant,75,2790,751,685,1436,0.523,583,1029,0.567,168,239,407,0.413,362,423,41,454,495,378,69,91,244,133,2032,290,7
150,Jayson Tatum,74,2648,672,754,1426,0.471,443,817,0.542,229,380,609,0.376,414,497,67,534,601,364,75,43,188,145,1987,608,8
149,De'Aaron Fox,74,2659,720,829,1549,0.465,506,969,0.522,214,366,580,0.369,312,423,67,273,340,418,151,31,194,193,1966,233,9
20,Stephen Curry,74,2420,650,795,1445,0.45,293,569,0.515,357,519,876,0.408,299,324,37,293,330,379,54,28,210,117,1956,189,10


In [283]:
#Get Player Names of the Top Category we have selected
names=top.PLAYER_NAME.to_list()

In [284]:
#Get the DF with Player Names from the top category
df_top=df_bq2[df_bq2['PLAYER_NAME'].isin(names)]

In [285]:
#Get Trended Data Point 

In [286]:
fig = px.scatter(
  df_top, 
  x="Game Number", 
  y="Total {}".format(sort_category),
  color='PLAYER_NAME',
  color_discrete_sequence=px.colors.qualitative.G10,
  title="<b>Regular Season All Time Trended Player {} vs. Game Number</b>".format(sort_category),
  template="plotly_white",
  hover_data=['PLAYER_NAME', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'Season',
       'Game Number'], 
  height=800, 
  width=1200,
  )

fig.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig.update_traces(marker={'size': 3})

fig

In [ ]:
fig.write_html("Regular Season All Time Trended Player {} vs. Game Number.html".format(sort_category),full_html=False, include_plotlyjs='cdn')
# fig.write_html("Trended Player {} vs. Game Number For The {} Season.html".format(sort_category,current_year),full_html=False, include_plotlyjs='cdn')

In [230]:
#POLAR CHARTS BY TOTALS

In [287]:
polar=px.bar_polar(
top, 
r=sort_category, 
theta="PLAYER_NAME", 
color="PLAYER_NAME",
template='ggplot2',
color_discrete_sequence=px.colors.qualitative.G10,
 title="Regular Season All Time Leaders In Total {}".format(sort_category),
 hover_data=['PLAYER_NAME', 'MIN', 'FGM', 'FGMX', 'FGA', 'FG_PCT', 'FG2M', 'FG2A',
       'FG2_PCT', 'FG3M', 'FG3MX', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS',
       'RK_Category'],
 height=800, width=1200,
                  )
polar.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)

In [ ]:
polar.write_html("Regular Season All Time Leaders In Total {}.html".format(sort_category),full_html=False, include_plotlyjs='cdn')
# polar.write_html("Leaders In Total {} For The {} Season.html".format(sort_category, current_year),full_html=False, include_plotlyjs='cdn')

In [ ]:
#2 variable scatter plot

In [288]:
cat1="GP"
cat2="AST"

fig_scatter = px.scatter(
  df_totals, 
  x=cat1, 
  y=cat2,
  color='PLAYER_NAME',
  color_discrete_sequence=px.colors.qualitative.G10,
  title="<b>Regular Season All Time Player {} vs. {}</b>".format(cat1,cat2),
  template="plotly_white",
  hover_data=['PLAYER_NAME', 'GP', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS'], 
  height=800, 
  width=1200,
  )

fig_scatter.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_scatter.update_traces(marker={'size': 10})

fig_scatter

In [ ]:
fig_scatter.write_html("Regular Season All Time Player {} vs. {}.html".format(cat1,cat2),full_html=False, include_plotlyjs='cdn')
# fig_scatter.write_html("Player {} vs. {} For The {} Season.html".format(cat1,cat2, current_year),full_html=False, include_plotlyjs='cdn')

In [ ]:
#MULTI METRIC PLOTS TOGETHER

In [ ]:
sort_categories=['PTS', 'REB','DREB', 'OREB', 'AST', 'STL', 'BLK','TOV', 'PF', 'FGM', 'FGA', 'FG3M',
       'FG3A', 'FGMX', 'FG3MX', 'FTM',  'FTA','MIN']

In [ ]:
#TRENDED

In [244]:
for i in sort_categories:
    df_totals['RK_Category']=df_totals[i].rank(method="dense", ascending=False)
    df_totals['RK_Category']=df_totals['RK_Category'].astype(int)
    top=df_totals.sort_values(by=i, ascending=False).head(10)
    names=top.PLAYER_NAME.to_list()
    df_top=df_bq2[df_bq2['PLAYER_NAME'].isin(names)]

    fig_multi = px.scatter(
    df_top, 
    x="Game Number", 
    y="Total {}".format(i),
    color='PLAYER_NAME',
    color_discrete_sequence=px.colors.qualitative.G10,
    title="<b>Regular Season All Time Trended Player {} vs. Game Number</b>".format(i),
    template="plotly_white",
    hover_data=['PLAYER_NAME','Total PTS', 'Total REB',
       'Total DREB', 'Total OREB', 'Total AST', 'Total STL', 'Total BLK',
       'Total TOV', 'Total PF', 'Total FGM', 'Total FGA', 'Total FG3M',
       'Total FG3A', 'Total FGMX', 'Total FG3MX', 'Total FTM', 'Total FTA',
       'Total MIN'
        #          'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
        # 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'Season',
        # 'Game Number'
        ], 
    height=800, 
    width=1200,
    )

    fig_multi.update_layout(
        title_x=0.5,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=20)),
        xaxis = dict(tickfont = dict(size=20)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_multi.update_traces(marker={'size': 3})
    fig_multi.write_html("Regular Season All Time Trended Player {} vs. Game Number.html".format(i),full_html=False, include_plotlyjs='cdn')
    # fig_multi.write_html("Trended Player {} vs. Game Number For The {} Season.html".format(i, current_year),full_html=False, include_plotlyjs='cdn')    

In [ ]:
#POLAR

In [ ]:
for i in sort_categories:
    df_totals['RK_Category']=df_totals[i].rank(method="dense", ascending=False)
    df_totals['RK_Category']=df_totals['RK_Category'].astype(int)
    top=df_totals.sort_values(by=i, ascending=False).head(10)
    names=top.PLAYER_NAME.to_list()
    df_top=df_bq2[df_bq2['PLAYER_NAME'].isin(names)]
    
    polar_multi=px.bar_polar(
    top, 
    r=sort_category, 
    theta="PLAYER_NAME", 
    color="PLAYER_NAME",
    template='ggplot2',
    color_discrete_sequence=px.colors.qualitative.G10,
    title="Regular Season All Time Leaders In Total {}".format(i),
    hover_data=['PLAYER_NAME', 'MIN', 'FGM', 'FGMX', 'FGA', 'FG_PCT', 'FG2M', 'FG2A',
        'FG2_PCT', 'FG3M', 'FG3MX', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
        'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS',
        'RK_Category'],
    height=800, width=1200,
                    )
    polar_multi.update_layout(
        title_x=0.5,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=20)),
        xaxis = dict(tickfont = dict(size=20)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    polar_multi.write_html("Regular Season All Time Leaders In Total {}.html".format(i),full_html=False, include_plotlyjs='cdn')
    # polar_multi.write_html("Leaders In Total {} For The {} Season.html".format(i, current_year),full_html=False, include_plotlyjs='cdn')    

In [ ]:
# PLAYOFFS

In [289]:
#Change the date to the previous day
query = """
SELECT * FROM `perceptive-ivy-290216.nba_api_player_boxscore.playoff_player_box_96_onwards`
"""
project_id = "perceptive-ivy-290216"
df_bq_playoffs = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [294]:
#USE FOR ALL TIME DATA
df_bq2_playoffs=df_bq_playoffs.copy()

#USE FOR SINGLE SEASON PLOTS
# current_year_playoffs='2023-24'
# df_bq2=df_bq_reg_season[df_bq_reg_season["Season"]==current_year_playoffs].copy()

df_bq2_playoffs=df_bq2_playoffs.fillna(0)
df_bq2_playoffs.head()

,SEASON_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,Season
0,41996,302,Mookie Blaylock,1610612737,ATL,Atlanta Hawks,0049600051,1997-05-10,ATL vs. CHI,L,39,3,10,0.3,1,6,0.167,3,4,0.75,0,2,2,6,1,0,4,2,10,-11,0,1996-97
1,41996,302,Mookie Blaylock,1610612737,ATL,Atlanta Hawks,0049600015,1997-04-27,ATL vs. DET,L,44,3,12,0.25,1,6,0.167,0,0,0.0,2,7,9,7,4,0,2,2,7,-8,0,1996-97
2,41996,302,Mookie Blaylock,1610612737,ATL,Atlanta Hawks,0049600047,1997-05-08,ATL @ CHI,W,46,9,13,0.692,8,9,0.889,0,0,0.0,0,7,7,9,3,0,5,1,26,10,0,1996-97
3,41996,302,Mookie Blaylock,1610612737,ATL,Atlanta Hawks,0049600020,1997-04-29,ATL @ DET,L,45,5,13,0.385,3,10,0.3,3,5,0.6,2,2,4,10,1,1,1,1,16,-12,0,1996-97
4,41996,302,Mookie Blaylock,1610612737,ATL,Atlanta Hawks,0049600030,1997-05-02,ATL @ DET,W,43,5,14,0.357,0,6,0.0,0,0,0.0,0,10,10,9,1,0,4,2,10,19,0,1996-97


In [295]:
df_bq2_playoffs['FGMX']=df_bq2_playoffs['FGA']-df_bq2_playoffs['FGM']
df_bq2_playoffs['FG3MX']=df_bq2_playoffs['FG3A']-df_bq2_playoffs['FG3M']
df_bq2_playoffs['FG2M']=df_bq2_playoffs['FGM']-df_bq2_playoffs['FG3M']
df_bq2_playoffs['FG2A']=df_bq2_playoffs['FGA']-df_bq2_playoffs['FG3A']
df_bq2_playoffs['FG2_PCT']=df_bq2_playoffs['FG2M']/df_bq2_playoffs['FG2A']
df_bq2_playoffs=df_bq2_playoffs.round({'FG2_PCT': 3})

In [296]:
#Add Game Number for trended values plotting
df_bq2_playoffs["Game Number"] = df_bq2_playoffs.groupby("PLAYER_NAME")["GAME_DATE"].rank(method="dense", ascending=True)
df_bq2_playoffs["Game Number"]=df_bq2_playoffs["Game Number"].astype(int)

In [297]:
#Get cumulative stat for each player
df_bq2_playoffs['Total PTS'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['PTS'].cumsum()
df_bq2_playoffs['Total REB'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['REB'].cumsum()
df_bq2_playoffs['Total DREB'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['DREB'].cumsum()
df_bq2_playoffs['Total OREB'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['OREB'].cumsum()
df_bq2_playoffs['Total AST'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['AST'].cumsum()
df_bq2_playoffs['Total STL'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['STL'].cumsum()
df_bq2_playoffs['Total BLK'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['BLK'].cumsum()
df_bq2_playoffs['Total TOV'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['TOV'].cumsum()
df_bq2_playoffs['Total PF'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['PF'].cumsum()
df_bq2_playoffs['Total FGM'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['FGM'].cumsum()
df_bq2_playoffs['Total FGA'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['FGA'].cumsum()
df_bq2_playoffs['Total FG3M'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['FG3M'].cumsum()
df_bq2_playoffs['Total FG3A'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['FG3A'].cumsum()
df_bq2_playoffs['Total FGMX'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['FGMX'].cumsum()
df_bq2_playoffs['Total FG3MX'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['FG3MX'].cumsum()
df_bq2_playoffs['Total FTM'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['FTM'].cumsum()
df_bq2_playoffs['Total FTA'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['FTA'].cumsum()
df_bq2_playoffs['Total MIN'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['MIN'].cumsum()

In [298]:
#Get Totals For Each Category
df_totals_playoffs=pd.DataFrame(df_bq2_playoffs.groupby(by=['PLAYER_ID','PLAYER_NAME'])[['MIN', 'FGM',
       'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA','OREB', 'DREB', 'REB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS','PLUS_MINUS','FGMX','FG3MX','FG2M','FG2A']].sum()).reset_index()
#Get Game Counts for Each Player
df_count_playoffs=pd.DataFrame(df_bq2_playoffs.groupby(by=['PLAYER_ID','PLAYER_NAME'])[["GAME_ID"]].count()).reset_index()
df_count_playoffs.columns=["PLAYER_ID","PLAYER_NAME","GP"]
df_totals_playoffs=df_totals_playoffs.merge(df_count_playoffs, left_on=["PLAYER_NAME","PLAYER_ID"],right_on=["PLAYER_NAME","PLAYER_ID"])
df_totals_playoffs.head()

,PLAYER_ID,PLAYER_NAME,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,FGMX,FG3MX,FG2M,FG2A,GP
0,2,Byron Scott,5361,934,1937,134,339,449,548,136,400,536,392,226,30,266,447,2451,69,1003,205,800,1598,183
1,3,Grant Long,1175,132,321,7,35,102,135,95,148,243,59,41,10,66,104,373,-24,189,28,125,286,42
2,7,Dan Schayes,1412,189,365,0,2,163,198,108,223,331,76,26,47,98,227,541,25,176,2,189,363,69
3,9,Sedale Threatt,1523,251,555,21,79,84,103,22,88,110,251,73,7,84,162,607,-1,304,58,230,476,70
4,12,Chris King,7,0,1,0,0,0,2,0,0,0,0,1,0,0,1,0,0,1,0,0,1,2


In [299]:
#Add additional columns
df_totals_playoffs['FG_PCT']=df_totals_playoffs['FGM']/df_totals_playoffs['FGA']
df_totals_playoffs['FG3_PCT']=df_totals_playoffs['FG3M']/df_totals_playoffs['FG3A']
df_totals_playoffs['FG2_PCT']=df_totals_playoffs['FG2M']/df_totals_playoffs['FG2A']
df_totals_playoffs=df_totals_playoffs.round({'FG_PCT': 3,'FG3_PCT': 3,'FG2_PCT': 3})

In [300]:
#Re-order the columns
df_totals_playoffs=df_totals_playoffs[['PLAYER_NAME', 'GP', 'MIN', 'FGM', 'FGMX', 'FGA','FG_PCT', 'FG2M', 'FG2A','FG2_PCT','FG3M', 
          'FG3MX', 'FG3A', 'FG3_PCT', 'FTM', 'FTA','OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]
# df_totals_playoffs["MIN"]=df_totals_playoffs["MIN"].apply(lambda x : "{:,}".format(x))
df_totals_playoffs.head()

,PLAYER_NAME,GP,MIN,FGM,FGMX,FGA,FG_PCT,FG2M,FG2A,FG2_PCT,FG3M,FG3MX,FG3A,FG3_PCT,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS
0,Byron Scott,183,5361,934,1003,1937,0.482,800,1598,0.501,134,205,339,0.395,449,548,136,400,536,392,226,30,266,447,2451,69
1,Grant Long,42,1175,132,189,321,0.411,125,286,0.437,7,28,35,0.2,102,135,95,148,243,59,41,10,66,104,373,-24
2,Dan Schayes,69,1412,189,176,365,0.518,189,363,0.521,0,2,2,0.0,163,198,108,223,331,76,26,47,98,227,541,25
3,Sedale Threatt,70,1523,251,304,555,0.452,230,476,0.483,21,58,79,0.266,84,103,22,88,110,251,73,7,84,162,607,-1
4,Chris King,2,7,0,1,1,0.0,0,1,0.0,0,0,0,NaN,0,2,0,0,0,0,1,0,0,1,0,0


In [311]:
sort_category_playoffs="PTS"

In [312]:
#Sort by a category and then rank it. This will help get the Top Players in a category
df_totals_playoffs['RK_Category']=df_totals_playoffs[sort_category_playoffs].rank(method="dense", ascending=False)
df_totals_playoffs['RK_Category']=df_totals_playoffs['RK_Category'].astype(int)
top_playoffs=df_totals_playoffs.sort_values(by=sort_category_playoffs, ascending=False).head(10)
top_playoffs

,PLAYER_NAME,GP,MIN,FGM,FGMX,FGA,FG_PCT,FG2M,FG2A,FG2_PCT,FG3M,FG3MX,FG3A,FG3_PCT,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,RK_Category
699,LeBron James,287,11863,2928,2968,5896,0.497,2458,4481,0.549,470,945,1415,0.332,1836,2479,430,2153,2583,2067,483,275,1034,655,8162,1324,1
317,Michael Jordan,179,7453,2188,2309,4497,0.487,2040,4051,0.504,148,298,446,0.332,1463,1766,305,847,1152,1022,376,158,546,541,5987,298,2
783,Kareem Abdul-Jabbar,237,8842,2356,2066,4422,0.533,2356,4418,0.533,0,4,4,0.0,1050,1419,416,1039,2481,767,179,444,427,798,5762,0,3
378,Kobe Bryant,220,8621,2014,2485,4499,0.448,1722,3617,0.476,292,590,882,0.331,1320,1617,230,889,1119,1040,310,144,647,660,5640,558,4
218,Shaquille O'Neal,216,8058,2041,1586,3627,0.563,2041,3627,0.563,0,0,0,NaN,1168,2317,866,1642,2508,582,117,459,649,769,5250,637,5
440,Tim Duncan,251,9351,1975,1964,3939,0.501,1970,3904,0.505,5,30,35,0.143,1217,1766,778,2081,2859,764,168,568,633,701,5172,1092,6
2097,Kevin Durant,170,6896,1684,1844,3528,0.477,1318,2499,0.527,366,663,1029,0.356,1251,1441,129,1198,1327,709,172,204,550,427,4985,643,7
142,Karl Malone,193,7881,1743,2025,3768,0.463,1737,3731,0.466,6,31,37,0.162,1269,1725,502,1560,2062,610,258,135,550,662,4761,125,8
1925,Jerry West,153,5355,1622,1802,3424,0.474,1622,3424,0.474,0,0,0,NaN,1211,1518,0,2,839,890,0,0,0,449,4455,0,9
642,Tony Parker,226,7765,1613,1888,3501,0.461,1494,3116,0.479,119,266,385,0.309,700,958,108,558,666,1143,198,19,594,429,4045,582,10


In [313]:
#Get Player Names of the Top Category we have selected
names_playoffs=top_playoffs.PLAYER_NAME.to_list()

In [317]:
#Get the DF with Player Names from the top category
df_top_playoffs=df_bq2_playoffs[df_bq2_playoffs['PLAYER_NAME'].isin(names_playoffs)]

In [318]:
#Get Trended Data Point 

In [319]:
fig_playoffs = px.scatter(
  df_top_playoffs, 
  x="Game Number", 
  y="Total {}".format(sort_category_playoffs),
  color='PLAYER_NAME',
  color_discrete_sequence=px.colors.qualitative.G10,
  title="<b>Playoffs All Time Trended Player {} vs. Game Number</b>".format(sort_category),
  template="plotly_white",
  hover_data=['PLAYER_NAME', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'Season',
       'Game Number'], 
  height=800, 
  width=1200,
  )

fig_playoffs.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_playoffs.update_traces(marker={'size': 3})

fig_playoffs

In [ ]:
fig_playoffs.write_html("Playoffs All Time Trended Player {} vs. Game Number.html".format(sort_category_playoffs),full_html=False, include_plotlyjs='cdn')
# fig_playoffs.write_html("Trended Player {} vs. Game Number For The {} Playoffs.html".format(sort_category_playoffs,current_year_playoffs),full_html=False, include_plotlyjs='cdn')

In [ ]:
#POLAR CHARTS BY TOTALS

In [307]:
polar_playoffs=px.bar_polar(
top_playoffs, 
r=sort_category, 
theta="PLAYER_NAME", 
color="PLAYER_NAME",
template='ggplot2',
color_discrete_sequence=px.colors.qualitative.G10,
 title="Playoffs All Time Leaders In Total {}".format(sort_category),
 hover_data=['PLAYER_NAME', 'MIN', 'FGM', 'FGMX', 'FGA', 'FG_PCT', 'FG2M', 'FG2A',
       'FG2_PCT', 'FG3M', 'FG3MX', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS',
       'RK_Category'],
 height=800, width=1200,
                  )
polar_playoffs.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)

In [ ]:
polar_playoffs.write_html("Playoffs All Time Leaders In Total {}.html".format(sort_category_playoffs),full_html=False, include_plotlyjs='cdn')
# polar_playoffs.write_html("Leaders In Total {} For The {} Playoffs.html".format(sort_category_playoffs, current_year_playoffs),full_html=False, include_plotlyjs='cdn')

In [ ]:
#2 variable scatter plot

In [308]:
cat1_playoffs="GP"
cat2_playoffs="AST"

fig_scatter_playoffs = px.scatter(
  df_totals_playoffs, 
  x=cat1, 
  y=cat2,
  color='PLAYER_NAME',
  color_discrete_sequence=px.colors.qualitative.G10,
  title="<b>Playoffs All Time Player {} vs. {}</b>".format(cat1,cat2),
  template="plotly_white",
  hover_data=['PLAYER_NAME', 'GP', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS'], 
  height=800, 
  width=1200,
  )

fig_scatter_playoffs.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_scatter_playoffs.update_traces(marker={'size': 10})

fig_scatter_playoffs

In [ ]:
fig_scatter_playoffs.write_html("Playoffs All Time Player {} vs. {}.html".format(cat1_playoffs,cat2_playoffs),full_html=False, include_plotlyjs='cdn')
# fig_scatter_playoffs.write_html("Player {} vs. {} For The {} Playoffs.html".format(cat1_playoffs,cat2_playoffs, current_year_playoffs),full_html=False, include_plotlyjs='cdn')

In [ ]:
#MULTI METRIC PLOTS TOGETHER

In [ ]:
sort_categories_playoffs=['PTS', 'REB','DREB', 'OREB', 'AST', 'STL', 'BLK','TOV', 'PF', 'FGM', 'FGA', 'FG3M',
       'FG3A', 'FGMX', 'FG3MX', 'FTM',  'FTA','MIN']

In [ ]:
#TRENDED

In [ ]:
for i in sort_categories_playoffs:
    df_totals_playoffs['RK_Category']=df_totals_playoffs[i].rank(method="dense", ascending=False)
    df_totals_playoffs['RK_Category']=df_totals_playoffs['RK_Category'].astype(int)
    top_playoffs=df_totals_playoffs.sort_values(by=i, ascending=False).head(10)
    names_playoffs=top_playoffs.PLAYER_NAME.to_list()
    df_top_playoffs=df_bq2_playoffs[df_bq2_playoffs['PLAYER_NAME'].isin(names)]

    fig_multi_playoffs = px.scatter(
    df_top_playoffs, 
    x="Game Number", 
    y="Total {}".format(i),
    color='PLAYER_NAME',
    color_discrete_sequence=px.colors.qualitative.G10,
    title="<b>Playoffs All Time Trended Player {} vs. Game Number</b>".format(i),
    template="plotly_white",
    hover_data=['PLAYER_NAME','Total PTS', 'Total REB',
       'Total DREB', 'Total OREB', 'Total AST', 'Total STL', 'Total BLK',
       'Total TOV', 'Total PF', 'Total FGM', 'Total FGA', 'Total FG3M',
       'Total FG3A', 'Total FGMX', 'Total FG3MX', 'Total FTM', 'Total FTA',
       'Total MIN'
        #          'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
        # 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'Season',
        # 'Game Number'
        ], 
    height=800, 
    width=1200,
    )

    fig_multi_playoffs.update_layout(
        title_x=0.5,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=20)),
        xaxis = dict(tickfont = dict(size=20)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_multi_playoffs.update_traces(marker={'size': 3})
    fig_multi_playoffs.write_html("Playoffs All Time Trended Player {} vs. Game Number.html".format(i),full_html=False, include_plotlyjs='cdn')
    # fig_multi_playoffs.write_html("Trended Player {} vs. Game Number For The {} Playoffs.html".format(i, current_year_playoffs),full_html=False, include_plotlyjs='cdn')    

In [ ]:
#POLAR

In [ ]:
for i in sort_categories_playoffs:
    df_totals_playoffs['RK_Category']=df_totals_playoffs[i].rank(method="dense", ascending=False)
    df_totals_playoffs['RK_Category']=df_totals_playoffs['RK_Category'].astype(int)
    top_playoffs=df_totals_playoffs.sort_values(by=i, ascending=False).head(10)
    names_playoffs=top_playoffs.PLAYER_NAME.to_list()
    df_top_playoffs=df_bq2_playoffs[df_bq2_playoffs['PLAYER_NAME'].isin(names)]
    
    polar_multi_playoffs=px.bar_polar(
    top_playoffs, 
    r=i, 
    theta="PLAYER_NAME", 
    color="PLAYER_NAME",
    template='ggplot2',
    color_discrete_sequence=px.colors.qualitative.G10,
    title="Playoffs All Time Leaders In Total {}".format(i),
    hover_data=['PLAYER_NAME', 'MIN', 'FGM', 'FGMX', 'FGA', 'FG_PCT', 'FG2M', 'FG2A',
        'FG2_PCT', 'FG3M', 'FG3MX', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
        'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS',
        'RK_Category'],
    height=800, width=1200,
                    )
    polar_multi_playoffs.update_layout(
        title_x=0.5,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=20)),
        xaxis = dict(tickfont = dict(size=20)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    polar_multi_playoffs.write_html("Playoffs All Time Leaders In Total {}.html".format(i),full_html=False, include_plotlyjs='cdn')
    # polar_multi_playoffs.write_html("Leaders In Total {} For The {} Playoffs.html".format(i, current_year_playoffs),full_html=False, include_plotlyjs='cdn')    